# Image classification model

We will start with creating an image classification model which we will later use to build an object detection model. At this stage we will use tensorflow.

Useful links:


In [1]:
# Imports
import tensorflow as tf
#import numpy as np
#import pandas as pd
#from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.models import Sequential
#import os

ImportError: cannot import name '_message' from 'google.protobuf.pyext' (/Users/sofyamalashchenko/Documents/Sofya/Uni stuff/Waterloo Winter 2022/Ingredient-detection/venv/lib/python3.8/site-packages/google/protobuf/pyext/__init__.py)